In [1]:
import surprise 

print(surprise.__version__)

1.1.0


## 1. 내장된 무비렌즈 데이터로 surprise 패키지 사용해보기

In [2]:
from surprise import SVD      # 행렬 분해 알고리즘 
from surprise import Dataset  # 내장 데이터 불러오는 라이브러리
from surprise import accuracy # accuracy의 rmse메소드로 예측 에러 평가
from surprise.model_selection import train_test_split  # 학습셋, 테스트셋 분리

#### 내장 데이터 로드하고 데이터를 학습셋, 테스트셋으로 분리

In [3]:
# 영화 데이터 10만개가 들어있는 "ml-100k" 데이터 로드하기
data = Dataset.load_builtin('ml-100k') 

# 로드된 데이터를 학습셋, 테스트셋(25%)으로 나누기
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

#### 행렬 분해 알고리즘으로 SVD 객체를 생성하고 학습 수행

In [4]:
algo = SVD()
algo.fit(trainset) 

#### 테스트 데이터에 예상 평점 데이터 예측하기

In [5]:
# 테스트 데이터 사용자 아이템에 대한 예상 평점 리스트 구하기
predictions = algo.test( testset )

print('prediction type :',type(predictions), ' size:',len(predictions))
print('25000개 예상 평점 리스트의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
25000개 예상 평점 리스트의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.6541091830455192, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=4.001627605321416, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9858122423489935, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.274190472307719, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.5715965464359294, details={'was_impossible': False})]

In [6]:
# 간단하게 확인해보기
[(pred.uid, pred.iid, pred.r_ui, pred.est) for pred in predictions[:3] ]

[('120', '282', 4.0, 3.6541091830455192),
 ('882', '291', 4.0, 4.001627605321416),
 ('535', '507', 5.0, 3.9858122423489935)]

In [7]:
# 예측 평점들의 에러인 rmse를 구해보면,
accuracy.rmse(predictions)

RMSE: 0.9490


0.9490230033197785

#### predict 메소드는 개별 사용자, 아이템에 대한 예측 평점 계산

In [8]:
# 주의 : user_id, item_id는 문자열 type으로 입력해야 함.
uid = str(196)
iid = str(302)

pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.13   {'was_impossible': False}


## 2. 실제 영화 평점 데이터로 surprise 패키지 사용해보기

In [9]:
from surprise import SVD      # 행렬 분해 알고리즘 
from surprise import Dataset  # 내장 데이터 불러오는 라이브러리
from surprise import accuracy # rmse로 예측 에러 평가
from surprise.model_selection import train_test_split  # 학습셋, 테스트셋 분리

#### Surprise 주요 모듈 소개

In [10]:
import pandas as pd

# 원본 csv 파일은 hearder가 있는데, surprise를 적용하려면 header를 없애줘야 한다.
ratings = pd.read_csv('./ml-latest-small/ratings.csv')

In [11]:
print(ratings.shape)
ratings.head()

(20000263, 4)


,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [12]:
%%time
# ratings_noh.csv 파일로 저장하면 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)

Wall time: 1min 58s


#### 데이터 로딩하기(load_from_file이용, reader로 파일 포멧 지정)

In [13]:
%%time
from surprise import Reader
# reader로 파일 포멧 지정하기( 컬럼 명 4개 지정, 콤마로 구분, 평점 범위는 0.5 ~ 5점 )
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))

# 데이터 로딩하기
data=Dataset.load_from_file('./ml-latest-small/ratings_noh.csv',reader=reader)

Wall time: 38.5 s


In [14]:
%%time
# 데이터를 학습셋, 테스트셋으로 나누기
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Wall time: 51.5 s


In [15]:
%%time
# SVD 알고리즘 적용(잠재 요인의 차원 수를 50개, 수행시마다 동일한 결과 도출을 위해 random_state 설정)
algo = SVD(n_factors=50, random_state=0)

Wall time: 3.99 ms


In [14]:
%%time 
# 학습셋으로 학습
algo.fit(trainset)

Wall time: 8min 51s


In [ ]:
%%time
# 테스트셋으로 예상 평점 예측
predictions = algo.test(testset)

In [23]:
predictions[:5]

[Prediction(uid='83343', iid='1103', r_ui=3.0, est=3.38796319353521, details={'was_impossible': False}),
 Prediction(uid='7884', iid='2028', r_ui=4.0, est=4.1887959317428045, details={'was_impossible': False}),
 Prediction(uid='55906', iid='296', r_ui=1.0, est=3.8767479288301274, details={'was_impossible': False}),
 Prediction(uid='100224', iid='2762', r_ui=3.5, est=4.304327544487485, details={'was_impossible': False}),
 Prediction(uid='24636', iid='761', r_ui=2.0, est=2.7018162180654444, details={'was_impossible': False})]

In [24]:
%%time
# RMSE 평가
accuracy.rmse(predictions)

RMSE: 0.7895
Wall time: 4.35 s


0.7895061187724682

### 샘플링 후 판다스 dataframe 활용해서 학습, 예측, 평가 재 수행

In [31]:
%%time
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ml-latest-small/ratings.csv') 

print(ratings.shape)
ratings.head()

(20000263, 4)
Wall time: 15.8 s


##### 샘플링

In [32]:
# 전체 2천만 건의 데이터 중 1%(20만 건) 랜덤 샘플링
ratings = ratings.sample(frac=0.01)

print(ratings.shape)
ratings.head()

(200003, 4)


,userId,movieId,rating,timestamp
2945547,20011,8783,4.0,2007-02-24 20:35:41
3793874,25854,76,2.0,1997-04-29 21:02:52
18564293,128481,150,5.0,2002-06-01 17:21:55
13677338,94484,5952,3.0,2008-05-21 17:17:27
1435951,9705,29,1.0,1996-12-12 13:44:07


In [34]:
%%time
# 데이터 로드하기
reader = Reader(rating_scale=(0.5, 5.0))
# ratings DataFrame 에서 컬럼은 userId, itemId, rating 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# 데이터를 학습셋, 테스트셋으로 나누기
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 행렬 분해 알고리즘 학습 및 예측, 평가(RMSE)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.9412
Wall time: 52.4 s


## 3. Cross Validation(교차 검증)과 GridSearchCV(하이퍼 파라미터 튜닝)

In [35]:
%%time
from surprise.model_selection import cross_validate # 교차 검증 : 과적합 피한 모델 학습이 가능하다.

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('./ml-latest-small/ratings.csv') # reading data in pandas df

# 데이터 샘플링 (2천만 건 데이터를 20만 건으로)
ratings = ratings.sample(frac=0.01)

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# SVD 알고리즘 
algo = SVD(random_state=0) 

# 교차 검증
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9425  0.9452  0.9416  0.9490  0.9401  0.9437  0.0031  
MAE (testset)     0.7311  0.7346  0.7325  0.7369  0.7293  0.7329  0.0027  
Fit time          9.67    9.66    9.99    10.13   9.21    9.73    0.32    
Test time         0.25    0.48    0.24    0.24    0.23    0.29    0.10    
Wall time: 1min 9s


### 그리드 서치 CV 이용

In [36]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# GridSearchCV 세팅 : CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs

In [37]:
%%time
# GridSearchCV로 학습
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

# 20만 건 데이터 기준 실행시간 (원본 데이터 2천만 건이면 8분~10분 정도 걸림)

0.9451494212785595
{'n_epochs': 20, 'n_factors': 50}
Wall time: 7min 49s


## 4. Surprise 를 이용한 개인화 영화 추천 시스템 구축

In [ ]:
# # 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
# data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
# algo = SVD(n_factors=50, random_state=0)
# algo.fit(data)

### 전체 데이터로 학습 진행(학습셋, 테스트셋 나누지 않고)

In [17]:
%%time
from surprise.dataset import DatasetAutoFolds  # 데이터 세트 전체를 학습 데이터로 사용할 수 있게 해주는 라이브러리

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()
# 실행시간 3분

Wall time: 3min 27s


In [18]:
%%time
# SVD 협업필터링으로 추천모델 학습(하이퍼 파라미터는 앞서 그리드서치로 구한 것들)
algo = SVD(n_epochs=20, n_factors=50, random_state=0)

algo.fit(trainset)
# 실행시간 11분

Wall time: 9min 43s


In [19]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
41       42  Dead Presidents (1995)  Action|Crime|Drama


In [20]:
# predict 메소드를 사용해서 예측 평점 구하기
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 2.95   {'was_impossible': False}


### 특정 사용자가 아직 안 본 전체 영화 추출 후 예측 평점 순으로 영화 추천

In [21]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 35 추천대상 영화수: 27243 전체 영화수: 27278


In [22]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화(27243개)에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [24]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print("")
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 35 추천대상 영화수: 27243 전체 영화수: 27278

##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.403255960567584
Braveheart (1995) : 4.354730709188801
Pulp Fiction (1994) : 4.279467874657644
Shawshank Redemption, The (1994) : 4.268033622651485
Schindler's List (1993) : 4.218225009929055
Silence of the Lambs, The (1991) : 4.216492633386306
Godfather: Part II, The (1974) : 4.206242413033943
Lord of the Rings: The Return of the King, The (2003) : 4.204346268855151
Personal Journey with Martin Scorsese Through American Movies, A (1995) : 4.197629744293488
Frozen Planet (2011) : 4.197580124016751
